In [87]:
# Librerías
import sqlite3  # Módulo para conectar con la base de datos
import pandas as pd  # Módulo para trabajar con dataframes

# Conectar a la base de datos
conn = sqlite3.connect("data/db_movies")  # Conectamos con la base de datos
cur = conn.cursor()  # Creamos un cursor

# Consultamos las tablas de la base de datos
cur.execute("select name from sqlite_master where type='table'")
print(cur.fetchall())

[('ratings',), ('movies',), ('movies_clean',), ('full_table',), ('data_clean',)]


# Preprocesado y unión

In [83]:
# Consulta movies y colocamos en un dataframe
movies = pd.read_sql_query("SELECT * FROM movies", conn)
movies.info()

# Verificar si hay valores nulos
print("---------------------------------NULOS---------------------")
print(movies.isnull().sum())

# Buscar duplicados en title y ver coincidencias
print("------------------------------DUPLICADOS---------------------")
movies[movies.duplicated(subset=["title"], keep=False)].sort_values(by="title")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
---------------------------------NULOS---------------------
movieId    0
title      0
genres     0
dtype: int64
------------------------------DUPLICADOS---------------------


,movieId,title,genres
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Thriller
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
650,838,Emma (1996),Comedy|Drama|Romance
5601,26958,Emma (1996),Romance
5854,32600,Eros (2004),Drama
9135,147002,Eros (2004),Drama|Romance
2141,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller
5931,34048,War of the Worlds (2005),Action|Adventure|Sci-Fi|Thriller
6932,64997,War of the Worlds (2005),Action|Sci-Fi


Vemos que hay 5 peliculas que contienen valores duplicados, pero estos se diferencian en los generos, procedemos a buscar en páginas web de películas para verificar los generos de cada una de ellas y proceder a eliminar los duplicados.

In [77]:
import funciones as fn

# Ejecutamos el archivo sql
fn.ejecutar_sql("preprocessing.sql", cur)
# Consultamos las tablas de la base de datos
cur.execute("select name from sqlite_master where type='table' ")
# Mostramos las tablas
cur.fetchall()

[('ratings',),
 ('movies',),
 ('movies_clean',),
 ('full_table',),
 ('data_clean',)]

La función realiza: preprocesado de duplicados y unión de los datasets y depuración de peliculas y usuarios

* full_table (tabla sin depurar) se crea para analizar las gráficas y tomar decisiones de depurar
* tabla data_clean (tabla depurada) es la tabla final depurando peliculas y usuarios

In [66]:
# Consulta movies y colocamos en un dataframe
movies = pd.read_sql_query("SELECT * FROM movies_clean", conn)
movies.info()

# Verificar si hay valores nulos
print("---------------------------------NULOS---------------------")
print(movies.isnull().sum())

# Buscar duplicados en title y ver coincidencias
print("------------------------------DUPLICADOS---------------------")
movies[movies.duplicated(subset=["title"], keep=False)].sort_values(by="title")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9737 entries, 0 to 9736
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9737 non-null   int64 
 1   title    9737 non-null   object
 2   genres   9737 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.3+ KB
---------------------------------NULOS---------------------
movieId    0
title      0
genres     0
dtype: int64
------------------------------DUPLICADOS---------------------


,movieId,title,genres


# Tabla unida sin depurar peliculas ni usuarios

In [67]:
# Consulta movies y colocamos en un dataframe
df = pd.read_sql_query("SELECT * FROM full_table", conn)
df.info()

# Verificar si hay valores nulos
print("---------------------------------NULOS---------------------")
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100836 non-null  int64  
 1   title      100836 non-null  object 
 2   genres     100836 non-null  object 
 3   userId     100836 non-null  int64  
 4   rating     100836 non-null  float64
 5   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 4.6+ MB
---------------------------------NULOS---------------------
movieId      0
title        0
genres       0
userId       0
rating       0
timestamp    0
dtype: int64


In [68]:
###calcular la distribución de calificaciones y graficar con plotly y go
import plotly.express as px

# Calcular la distribución de calificaciones
tabla = pd.read_sql_query(
    """
    SELECT rating, COUNT(*) AS conteo
    FROM full_table
    GROUP BY rating
    ORDER BY rating
    """,
    conn,
)

# Graficar con plotly
fig = px.bar(tabla, x="rating", y="conteo", title="Distribución de calificaciones")
fig.show()

Vemos que el dataset no tiene calificaciones en 0, no hay que eliminar ninguna película por falta de calificaciones.

In [36]:
### calcular cada usuario cuátos peliculas calificó
rating_users = pd.read_sql(
    """ select "userId" as userId,
                         count(*) as cnt_rat
                         from full_table
                         group by "userId"
                         order by cnt_rat asc
                         """,
    conn,
)

fig = px.histogram(
    rating_users,
    x="cnt_rat",
    title="Hist frecuencia de numero de calificaciones por usario",
)
fig.show()


rating_users.describe()

,userId,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


In [55]:
#### filtrar usuarios con más de 25 películas calificados (para tener calificación confiable)
rating_users2 = pd.read_sql(
    """ select "userId" as userId,
                         count(*) as cnt_rat
                         from full_table
                         group by "userId"
                         having cnt_rat >=25
                         order by cnt_rat asc
                         """,
    conn,
)


### gráfica distribución después de filtrar datos
fig = px.histogram(
    rating_users2,
    x="cnt_rat",
    title="Hist frecuencia de numero de calificaciones por usuario",
)
fig.show()


### ver distribución después de filtros
rating_users2.describe()

,userId,cnt_rat
count,547.000000,547.000000
mean,303.956124,181.839122
std,176.291378,279.905223
min,1.000000,25.000000
25%,151.500000,43.500000
50%,306.000000,86.000000
75%,455.500000,190.500000
max,610.000000,2698.000000


* Se filtran los usuarios que han calificado más de 25 películas, ya que menos de ahí se considera que no es un usuario con experiencia en el cine.

* Se decide dejar los usuarios que han visto 2698 películas aunque son pocos, son datos lógicos, ya que se considera que es un número razonable de películas y puede ser un usuario con muchos años de experiencia en el cine. (Suponiendo 2698 peliculas cada una de 2 horas, viendo 3 a la semana = 2698*2(dur.pelicula)/3(semanas) = 1798 semanas = 1798/52 (semanas en un año) = 34.5 años)

In [59]:
#### verificar cuantas calificaciones tiene cada pelicula
rating_movies = pd.read_sql(
    """ select movieId as movieId ,
                         count(*) as cnt_rat
                         from full_table
                         group by "movieId"
                         order by cnt_rat desc
                         """,
    conn,
)

fig = px.histogram(
    rating_movies,
    x="cnt_rat",
    title="Hist frecuencia de numero de calificaciones para cada pelicula",
)
fig.show()

### analizar distribucion de calificaciones por libro
rating_movies.describe()

,movieId,cnt_rat
count,9719.000000,9719.000000
mean,42221.740714,10.375141
std,52177.716338,22.406220
min,1.000000,1.000000
25%,3243.500000,1.000000
50%,7294.000000,3.000000
75%,76519.500000,9.000000
max,193609.000000,329.000000


Vemos que sin el filtro hay 9719 peliculas que han sido calificadas, pero tener en cuenta que una pelicula es recomendada por pocas calificaciones, no es un buen criterio

In [62]:
####Filtrar libros que no tengan más de 30 calificaciones
rating_movies2 = pd.read_sql(
    """ select movieId ,
                         count(*) as cnt_rat
                         from full_table
                         group by "movieId"
                         having cnt_rat>=30
                         order by cnt_rat desc
                         """,
    conn,
)

fig = px.histogram(
    rating_movies2,
    x="cnt_rat",
    title="Hist frecuencia de numero de calificaciones para cada pelicula",
)
fig.show()

rating_movies2.describe()

,movieId,cnt_rat
count,882.000000,882.000000
mean,14538.518141,65.783447
std,27857.290961,42.543129
min,1.000000,30.000000
25%,1093.250000,39.000000
50%,2556.500000,50.000000
75%,6376.000000,77.000000
max,152081.000000,329.000000


Vemos que hay 882 peliculas que han sido calificadas mas de 30 veces

# Tabla final depurada

In [80]:
# Ver tabla data_clean
data_clean = pd.read_sql_query("SELECT * FROM data_clean", conn)
data_clean.info()
data_clean.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56219 entries, 0 to 56218
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    56219 non-null  int64  
 1   title      56219 non-null  object 
 2   genres     56219 non-null  object 
 3   userId     56219 non-null  int64  
 4   rating     56219 non-null  float64
 5   timestamp  56219 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 2.6+ MB


,movieId,title,genres,userId,rating,timestamp
32003,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi,489,1.0,1333101402
12116,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,7,2.5,1106635940
15267,849,Escape from L.A. (1996),Action|Adventure|Sci-Fi|Thriller,19,2.0,965704033
29587,2231,Rounders (1998),Drama,608,5.0,1117162030
42564,5299,My Big Fat Greek Wedding (2002),Comedy|Romance,169,4.5,1059427933
28568,2076,Blue Velvet (1986),Drama|Mystery|Thriller,577,3.0,945978437
22588,1307,When Harry Met Sally... (1989),Comedy|Romance,376,4.0,1364994443
22779,1356,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi|Thriller,217,4.0,955940625
52244,59315,Iron Man (2008),Action|Adventure|Sci-Fi,78,3.0,1252575270
15784,908,North by Northwest (1959),Action|Adventure|Mystery|Romance|Thriller,307,4.0,1198813463
